# 本模块实现特征衍生

In [110]:
import pandas as pd
import numpy as np
# base
import math

import pandas as pd
import numpy as np
import sys
import datetime
import re
# data process
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures, LabelBinarizer
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype
import seaborn as sns
# model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
# model test
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

## 训练样本和预测样本概况

In [111]:
# 训练样本 ：(1368146, 55)，总计15000个不同的用户，日期为2015-3-1至2016-2-23，不均匀分布,正负bi：14230:770
# data_path = "./data/train.tsv"
# data_id = "./data/train_id.tsv"
data_path = "./data/train.tsv"
data_id = "./data/train_id.tsv"
dat = pd.read_csv(data_path, sep='\t')
dat_id = pd.read_csv(data_id,sep="\t")

train_data  = pd.merge(dat,dat_id, how="inner", on="PERSONID")

print(train_data.shape)
train_data.head()

(1368146, 56)


,PERSONID,APPLYNO,FTR0,FTR1,FTR2,FTR3,FTR4,FTR5,FTR6,FTR7,...,FTR44,FTR45,FTR46,FTR47,FTR48,FTR49,FTR50,FTR51,CREATETIME,LABEL
0,8ba94305557a4ec72435a9eb78001350,20fab21284e57a4ccd3127553920e37b,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000000,0.050012,0.0,0.001532,A2B186C398E0D0,2015-09-14,0
1,8ba94305557a4ec72435a9eb78001350,73f32b8b6e199129abb8fe0cb6d6c867,0.033333,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.033333,0.0,0.0,0.000000,0.033333,0.0,0.005156,A24B176C1239E0D0,2015-09-14,0
2,8ba94305557a4ec72435a9eb78001350,324da7a0b1a5c1d3b321685d734a9dc8,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-09-14,0
3,8ba94305557a4ec72435a9eb78001350,341ceb4ae7b0ca1c46653f5a116d4bd2,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-10-16,0
4,8ba94305557a4ec72435a9eb78001350,ca260c60e62de319e38a475e2455afb8,0.050000,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.050000,0.0,0.0,0.000000,0.050000,0.0,0.005156,A24B176C1239E0D0,2015-10-16,0


In [112]:
# 需要预测的样本 ： (232502, 55)，总计2500个用户,日期为2015-3-1至2016-2-23，不均匀分布
data_path = "./data/test_A.tsv"
test_data = pd.read_csv(data_path,sep="\t")
test_data['LABEL'] = 2
test_data.head()


,PERSONID,APPLYNO,FTR0,FTR1,FTR2,FTR3,FTR4,FTR5,FTR6,FTR7,...,FTR44,FTR45,FTR46,FTR47,FTR48,FTR49,FTR50,FTR51,CREATETIME,LABEL
0,d7d27cd05dd10be17a9d5339bb1596ad,d6114afdd829c6a24553bdd48ebdbf78,0.050000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.050000,0.0,0.0,0.000023,0.050012,0.0,0.0,A0B0C6E0D0,2015-03-01,2
1,d7d27cd05dd10be17a9d5339bb1596ad,12da95c0ac1ff008196939b1706b9786,0.050000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.050000,0.0,0.0,0.001767,0.050012,0.0,0.0,A0B0C3E0D0,2015-03-01,2
2,d7d27cd05dd10be17a9d5339bb1596ad,1fe1085070c8a8eeb7ba3b7596c6b271,0.050000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.550000,0.0,0.0,0.000000,0.050000,0.0,0.0,"A2B307C46E0D0,A2B390C540E0D0,A36B257C339E0D0,A...",2015-03-01,2
3,d7d27cd05dd10be17a9d5339bb1596ad,2b9b3749b7b8d9e7c81814686bb1dba8,0.274993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.025007,0.0,0.0,0.000023,0.025000,0.0,0.0,A0B0C6E0D0,2015-05-31,2
4,d7d27cd05dd10be17a9d5339bb1596ad,a0e026f43eef17df34e65f5839d11fc2,0.050000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.050000,0.0,0.0,0.001767,0.050012,0.0,0.0,A0B0C3E0D0,2015-05-31,2


In [107]:
test_data.shape

(232502, 56)

In [132]:
data = pd.concat([train_data,test_data], axis =0)
print(data.shape)
data.head()

(1600648, 56)


,PERSONID,APPLYNO,FTR0,FTR1,FTR2,FTR3,FTR4,FTR5,FTR6,FTR7,...,FTR44,FTR45,FTR46,FTR47,FTR48,FTR49,FTR50,FTR51,CREATETIME,LABEL
0,8ba94305557a4ec72435a9eb78001350,20fab21284e57a4ccd3127553920e37b,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000000,0.050012,0.0,0.001532,A2B186C398E0D0,2015-09-14,0
1,8ba94305557a4ec72435a9eb78001350,73f32b8b6e199129abb8fe0cb6d6c867,0.033333,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.033333,0.0,0.0,0.000000,0.033333,0.0,0.005156,A24B176C1239E0D0,2015-09-14,0
2,8ba94305557a4ec72435a9eb78001350,324da7a0b1a5c1d3b321685d734a9dc8,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-09-14,0
3,8ba94305557a4ec72435a9eb78001350,341ceb4ae7b0ca1c46653f5a116d4bd2,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-10-16,0
4,8ba94305557a4ec72435a9eb78001350,ca260c60e62de319e38a475e2455afb8,0.050000,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.050000,0.0,0.0,0.000000,0.050000,0.0,0.005156,A24B176C1239E0D0,2015-10-16,0


## 特征处理

In [144]:
numeric_fea = ['FTR0', 'FTR1', 'FTR2', 'FTR3', 
               'FTR4', 'FTR5', 'FTR6', 'FTR7', 
               'FTR8', 'FTR9', 'FTR10', 'FTR11', 
               'FTR12', 'FTR13', 'FTR14', 'FTR15', 
               'FTR16', 'FTR17', 'FTR18', 'FTR19', 
               'FTR20', 'FTR21', 'FTR22', 'FTR23', 
               'FTR24', 'FTR25', 'FTR26', 'FTR27', 
               'FTR28', 'FTR29', 'FTR30', 'FTR31', 'FTR32',
               'FTR33', 'FTR34', 'FTR35', 'FTR36', 'FTR37',
               'FTR38', 'FTR39', 'FTR40', 'FTR41', 'FTR42', 'FTR43', 
               'FTR44', 'FTR45', 'FTR46', 'FTR47', 'FTR48', 'FTR49',
               'FTR50']

string_fea = ['PERSONID', 'APPLYNO', 'FTR51', 'CREATETIME']


### 数值型特征处理

In [155]:

for num_fea in  numeric_fea:
    minn = data[num_fea].quantile(0.01)
    maxx = data[num_fea].quantile(0.99)
    data.loc[data[num_fea] < minn, num_fea] = minn
    data.loc[data[num_fea] > maxx, num_fea] = maxx
    #log处理
    data[num_fea] = np.log(data[num_fea] + 1)




### 数值特征组合

In [162]:
# def deal_ftr51(ser):
    
#     return  
    

func_list = [np.mean, np.sum, np.std]
func_dict = {i : func_list for i in numeric_fea }   
func_dict['APPLYNO'] = ['count']
func_dict['FTR51'] = [pd.Series.nunique]
func_dict['LABEL'] = [np.unique]
df_agg = data.groupby("PERSONID").agg(func_dict)


In [168]:
df_51 = data[['PERSONID','APPLYNO','FTR51']]
df_51.head(30)

,PERSONID,APPLYNO,FTR51
0,8ba94305557a4ec72435a9eb78001350,20fab21284e57a4ccd3127553920e37b,A2B186C398E0D0
1,8ba94305557a4ec72435a9eb78001350,73f32b8b6e199129abb8fe0cb6d6c867,A24B176C1239E0D0
2,8ba94305557a4ec72435a9eb78001350,324da7a0b1a5c1d3b321685d734a9dc8,A0B0C6E0D0
3,8ba94305557a4ec72435a9eb78001350,341ceb4ae7b0ca1c46653f5a116d4bd2,A0B0C6E0D0
4,8ba94305557a4ec72435a9eb78001350,ca260c60e62de319e38a475e2455afb8,A24B176C1239E0D0
5,8ba94305557a4ec72435a9eb78001350,55eac334641dd47c43d5f95c5e2b6e6f,A2B186C398E0D0
6,8ba94305557a4ec72435a9eb78001350,ce6cf113cde7df53b776dafb77049487,A0B0C6E0D0
7,8ba94305557a4ec72435a9eb78001350,f699af5bb8589446c7a1d06632b35f7b,A24B3634C1239E0D0
8,8ba94305557a4ec72435a9eb78001350,b55e5b0bd3817ffb897e1dcaf430f3ca,A2B287C398E0D0
9,8ba94305557a4ec72435a9eb78001350,54d4bf98bd7baadd5306f41ab1e3af89,A24B3634C1239E0D0


In [164]:
df_agg.shape
df_agg.head()

FTR0                     FTR1            \
                                      mean       sum       std mean  sum  std   
PERSONID                                                                        
000e3ee6e04a8b18871e1c85ffab6e49  0.051196  4.146898  0.028574  0.0  0.0  0.0   
000eba50dc8c2e3899070c6b82986814  0.050606  5.111209  0.036453  0.0  0.0  0.0   
00139506cbd54b670e1e6dec234190e1  0.047762  1.862719  0.004575  0.0  0.0  0.0   
0014fdb3625079de4a0d2131fd860641  0.059731  9.138827  0.070888  0.0  0.0  0.0   
001de94e8d0d152e26ad87cb409003f9  0.053725  3.116060  0.051950  0.0  0.0  0.0   

                                 FTR2           FTR3  ...       FTR48 FTR49  \
                                 mean  sum  std mean  ...         std  mean   
PERSONID                                              ...                     
000e3ee6e04a8b18871e1c85ffab6e49  0.0  0.0  0.0  0.0  ...    0.009009   0.0   
000eba50dc8c2e3899070c6b82986814  0.0  0.0  0.0  0.0  ...    0.008315   0.0   
00139506cbd54b670e1e6dec234190e1  0.0  0.0  0.0  0.0  ...    0.077352   0.0   
0014fdb3625079de4a0d2131fd860641  0.0  0.0  0.0  0.0  ...    0.009840   0.0   
001de94e8d0d152e26ad87cb409003f9  0.0  0.0  0.0  0.0  ...    0.007916   0.0   

                                               FTR50                      \
                                  sum  std      mean       sum       std   
PERSONID                                                                   
000e3ee6e04a8b18871e1c85ffab6e49  0.0  0.0  0.000000  0.000000  0.000000   
000eba50dc8c2e3899070c6b82986814  0.0  0.0  0.000000  0.000000  0.000000   
00139506cbd54b670e1e6dec234190e1  0.0  0.0  0.000000  0.000000  0.000000   
0014fdb3625079de4a0d2131fd860641  0.0  0.0  0.000000  0.000000  0.000000   
001de94e8d0d152e26ad87cb409003f9  0.0  0.0  0.000138  0.008026  0.000306   

                                 APPLYNO   FTR51  LABEL  
                                   count nunique unique  
PERSONID                                                 
000e3ee6e04a8b18871e1c85ffab6e49      81      42      0  
000eba50dc8c2e3899070c6b82986814     101      50      0  
00139506cbd54b670e1e6dec234190e1      39      21      0  
0014fdb3625079de4a0d2131fd860641     153      51      2  
001de94e8d0d152e26ad87cb409003f9      58      31      0  

[5 rows x 156 columns]

In [165]:
colnames = [df_agg.columns.levels[0][i] + '_' + df_agg.columns.levels[1][j]
 for i,j in list(zip(df_agg.columns.labels[0], df_agg.columns.labels[1])) ]


In [166]:
df_agg.columns = colnames
df_agg.head()

,FTR0_mean,FTR0_sum,FTR0_std,FTR1_mean,FTR1_sum,FTR1_std,FTR2_mean,FTR2_sum,FTR2_std,FTR3_mean,...,FTR48_std,FTR49_mean,FTR49_sum,FTR49_std,FTR50_mean,FTR50_sum,FTR50_std,APPLYNO_count,FTR51_nunique,LABEL_unique
PERSONID,,,,,,,,,,,,,,,,,,,,,
000e3ee6e04a8b18871e1c85ffab6e49,0.051196,4.146898,0.028574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009009,0.0,0.0,0.0,0.000000,0.000000,0.000000,81,42,0
000eba50dc8c2e3899070c6b82986814,0.050606,5.111209,0.036453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.008315,0.0,0.0,0.0,0.000000,0.000000,0.000000,101,50,0
00139506cbd54b670e1e6dec234190e1,0.047762,1.862719,0.004575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.077352,0.0,0.0,0.0,0.000000,0.000000,0.000000,39,21,0
0014fdb3625079de4a0d2131fd860641,0.059731,9.138827,0.070888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009840,0.0,0.0,0.0,0.000000,0.000000,0.000000,153,51,2
001de94e8d0d152e26ad87cb409003f9,0.053725,3.116060,0.051950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.007916,0.0,0.0,0.0,0.000138,0.008026,0.000306,58,31,0


In [167]:
df_agg.to_csv('data/train_derive.csv')